# Модель ARIMA (пакет [`pmdarima`](https://alkaline-ml.com/pmdarima/index.html#))

Спецификация ARIMA(p,d,q) (ARIMA = AutoRegressive Integrated Moving Average)

$$
\begin{aligned}
	\Delta ^d y_t &= (\alpha_0+\alpha_1 t)+\sum_{j=1}^p\phi_j\Delta^d y_{t-j}+u_t+\sum_{s=1}^q\theta_s u_{t-s} &
	u_t&\sim WN(0,\sigma^2)
\end{aligned}
$$
где
* p - порядок авторегрессионной части
* d - порядок интегрирования
* q - порядок части скользящего среднего 
* $\alpha_0$ - константа (d=0) или снос (d>0)

Спецификация с использование лагового оператора

$$
\begin{aligned}
	\phi_p(L)(1-L)^dy_t&=(\alpha_0+\alpha_1t)+\theta_q(L)u_t & u_t&\sim WN(0,\sigma^2)
\end{aligned}
$$
где многочлены
$$
\begin{aligned}
	\phi_p(z)&=1-\phi_1z-\cdots-\phi_pz^p & \theta_q(z)&=1+\theta_1z+\cdots+\theta_qz^q
\end{aligned}
$$

In [ ]:
import numpy as np
import pandas as pd

import pmdarima as pm

import pandas_datareader.data as web

# настройки визуализации
import matplotlib.pyplot as plt

# Не показывать Warnings
import warnings
warnings.simplefilter(action='ignore', category=Warning)

Для подгонки используем класс [ARIMA](https://alkaline-ml.com/pmdarima/modules/generated/pmdarima.arima.ARIMA.html#pmdarima.arima.ARIMA)

При спецификации модели нужно задать параметры `order` (порядок модели) и  `trend`

|Параметры|`trend`|
|-|-|
|$\alpha_0=\alpha_1=0$|'n'|
|$\alpha_0\ne0,\alpha_1=0$|'c'|
|$\alpha_0,\alpha_1\ne0$|'ct'|


## Подгонка ARIMA заданного порядка

Загрузим из БД [`FRED`](https://fred.stlouisfed.org/) недельные данные по 3-Month Treasury Bill Secondary Market Rate (Symbol [`WTB3MS`](https://fred.stlouisfed.org/series/WTB3MS)) с 2000-01-01 по 2023-12-31 и создадим датафрейм `rate`

In [ ]:
y = web.DataReader(name='WTB3MS', data_source='fred', start='2000-01-01', end='2023-12-31')

Подгоним модель ARIMA(2,1,3) без сноса

Спецификация

$$
	(1-\phi_1L-\phi_2 L^2)(1-L) y_t=u_t+\theta_1 u_{t-1}+\theta_2 u_{t-2}+\theta_3 u_{t-3}
$$

In [ ]:
arima = pm.ARIMA(order=(2,1,3), trend='n')
arima.fit(y)
arima.summary()

In [ ]:
arima.plot_diagnostics()
plt.show()

## Прогнозирование

Построим прогноз  по модели ARIMA(1,1,1) со сносом

Спецификация

$$
	(1-\phi L)(1-L) y_t=\alpha_0+u_t+\theta u_{t-1}
$$

Численный прогноз на 10 периодов

In [ ]:
arima = pm.ARIMA(order=(1,1,1), trend='c')
# подгонка модели и прогноз
forecasts = arima.fit_predict(y, n_periods=10)
forecasts

Численный прогноз и доверительные интервалы

In [ ]:
forecasts, conf_int = arima.predict(n_periods=10, return_conf_int=True, alpha=0.05)
forecasts, conf_int

## Автоматический выбор порядка модели

используем метод [`auto_arima`](https://alkaline-ml.com/pmdarima/modules/generated/pmdarima.arima.auto_arima.html#pmdarima.arima.auto_arima)

Базовые параметры автоматического выбора

|Критерий|`auto_arima`|Значения|
|-|-|-|
|Информационный критерий|`information_criterion`|aic (по умолчанию), aicc, bib, hqic |
|тест единичного корня|`test`|kpss (по умолчанию), adf, pp|
|max порядок d|`max_d`| 2 по умолчанию |
|max порядок p,q|` max_p`, ` max_q`|5 по умолчанию|
|модель с сезонностью|`seasonal`|True по умолчанию|
|уровень значимость|`alpha`|0.05 по умолчанию|

__Замечание__: автоматические выбирается нужно ли включать снос/константу/тренд в модель


In [ ]:
arima_opt = pm.auto_arima(y, information_criterion='bic', seasonal=False)
arima_opt.get_params()

Прогноз по оптимальной модели

In [ ]:
arima_opt.predict(n_periods=10)